Download LEHD data at block level and join with block shapefile.

### Install / load packages

In [1]:
# pip install pygris

In [2]:
from pygris.data import get_lodes
from pygris import blocks
from pygris import places
import pandas as pd
import geopandas as gpd

C:\Users\jpg23\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\jpg23\anaconda3\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
C:\Users\jpg23\anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


### Load LEHD data using `get_lodes` function from `pygris`

In [3]:
states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "FL", "GA", "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]

In [4]:
lehd = []

for which_state in states:
    
    try:
    
        state_lehd = get_lodes(
            state = which_state, 
            year = 2019, # data from 2019
            version = "LODES8", # uses 2020 census blocks
            lodes_type = "wac", 
            agg_level = "block"
        )

        state_lehd["state"] = which_state

        lehd.append(state_lehd)
    
    except:
        
        pass

In [5]:
df = pd.concat(lehd)[["state", "w_geocode", "C000"]]

In [6]:
df

,state,w_geocode,C000
0,AL,010010201001000,2
1,AL,010010201001001,2
2,AL,010010201001002,1
3,AL,010010201001012,28
4,AL,010010201001019,15
...,...,...,...
8844,WY,560459513003039,3
8845,WY,560459513003040,1
8846,WY,560459513003043,5
8847,WY,560459513003044,67


In [7]:
df.state.unique()

array(['AL', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI', 'ID',
       'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN',
       'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH',
       'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA',
       'WA', 'WV', 'WI', 'WY'], dtype=object)

In [8]:
df.state.nunique()

48

### Load block group shapefiles (and land area)

In [ ]:
bl = []

for which_state in states:
    
    try:
    
        state_bl = blocks(state = which_state, year = 2020) # load 2020 boundaries to match LODES8 boundaries

        state_bl["state"] = which_state

        bl.append(state_bl)
    
    except:
        
        pass

Using FIPS code '01' for input 'AL'
Using FIPS code '02' for input 'AK'
Using FIPS code '04' for input 'AZ'
Using FIPS code '05' for input 'AR'
Using FIPS code '06' for input 'CA'
Using FIPS code '08' for input 'CO'
Using FIPS code '09' for input 'CT'
Using FIPS code '10' for input 'DE'
Using FIPS code '12' for input 'FL'
Using FIPS code '13' for input 'GA'
Using FIPS code '15' for input 'HI'
Using FIPS code '16' for input 'ID'
Using FIPS code '17' for input 'IL'
Using FIPS code '18' for input 'IN'
Using FIPS code '19' for input 'IA'
Using FIPS code '20' for input 'KS'
Using FIPS code '21' for input 'KY'
Using FIPS code '22' for input 'LA'


In [ ]:
bl_sf0 = pd.concat(bl)
bl_sf0.head()

In [ ]:
bl_sf = bl_sf0[["state", "GEOID10", "ALAND10", "geometry"]]

In [ ]:
bl_sf.head()

In [ ]:
bl_sf.state.unique()

In [ ]:
bl_sf.state.nunique()

In [ ]:
bl_sanfran = bl_sf.loc[bl_sf.GEOID10.str.contains('^06075'), :]

In [ ]:
bl_sanfran.plot()

### Join LEHD data and shapefile

In [ ]:
df.dtypes

In [ ]:
bl_sf.dtypes

In [ ]:
df.head()

In [ ]:
bl_sf.head()

In [ ]:
df.shape[0]

In [ ]:
bl_sf.shape[0]

In [ ]:
df[df['state'] == 'AR']

In [ ]:
bl_sf[bl_sf['state'] == 'AR'].head()

In [ ]:
bl_sf.dtypes

In [ ]:
df.dtypes

In [ ]:
final_df = bl_sf.merge(df, how = 'left', left_on = ['state', 'GEOID10'], right_on = ['state', 'w_geocode']).drop(columns = ['w_geocode']).rename(columns={"C000": "total_jobs", "ALAND10": "land_area", "GEOID10": "block"})

In [ ]:
final_df.shape[0]

In [ ]:
final_df.head()

In [ ]:
final_df[final_df['total_jobs'].notnull()].head()

In [ ]:
final_df.total_jobs.isnull().mean()

In [ ]:
final_df.state.nunique()

In [ ]:
df_states = df.state.unique().tolist()
final_df_states = final_df.state.unique().tolist()

list(set(df_states).difference(final_df_states))

In [ ]:
type(final_df)

### Subset to cities of interest

In [ ]:
city_list = [
  'Albuquerque NM',
  'Atlanta GA',
  'Austin TX',
  'Bakersfield CA',
  'Baltimore MD',
  'Boston MA',
  'Charlotte NC',
  'Chicago IL',
  'Cincinnati OH',
  'Cleveland OH',
  'Colorado Springs CO',
  'Columbus OH',
  'Dallas TX',
  'Denver CO',
  'Detroit MI',
  'El Paso TX',
  'Fort Worth TX',
  'Fresno CA',
  'Urban Honolulu HI',
  'Houston TX',
  'Indianapolis city (balance) IN',
  'Jacksonville FL',
  'Kansas City MO',
  'Las Vegas NV',
  'Los Angeles CA',
  'Louisville KY',
  'Memphis TN',
  'Miami FL',
  'Milwaukee WI',
  'Minneapolis MN',
  'Nashville-Davidson metropolitan government (balance) TN',
  'New Orleans LA',
  'New York NY',
  'Oakland CA',
  'Oklahoma City OK',
  'Omaha NE',
  'Orlando FL',
  'Philadelphia PA',
  'Phoenix AZ',
  'Pittsburgh PA',
  'Portland OR',
  'Raleigh NC',
  'Sacramento CA',
  'Salt Lake City UT',
  'San Antonio TX',
  'San Diego CA',
  'San Francisco CA',
  'San Jose CA',
  'Seattle WA',
  'St. Louis MO',
  'Tampa FL',
  'Tucson AZ',
  'Tulsa OK',
  'Washington DC']

In [ ]:
all_places = places(cb = True)

In [ ]:
all_places.head()

In [ ]:
pd.set_option('display.max_colwidth', None)
all_places.loc[all_places.NAME.str.contains('Nashville|Indianapolis|Honolulu|St\.? Louis'), :][['NAME', 'STUSPS']]

In [ ]:
all_places['city'] = all_places.apply(lambda x: '{} {}'.format(x['NAME'], x['STUSPS']), axis=1)

In [ ]:
all_places.city.head()

In [ ]:
all_places_simp = all_places[['city', 'geometry']]
the_places = all_places_simp[all_places_simp['city'].isin(city_list)]

In [ ]:
pd.reset_option('^display.', silent=True)
the_places.head()

In [ ]:
the_places.city.unique()

In [ ]:
the_places.city.nunique()

In [ ]:
in_places = the_places.city.unique().tolist()

list(set(city_list).difference(in_places))

In [ ]:
final_df.crs == the_places.crs

In [ ]:
df_joined = gpd.sjoin(final_df, the_places, how = "inner").drop(columns = ['index_right'])

In [ ]:
df_joined.head()

In [ ]:
sf_test = df_joined[df_joined['city']=='San Francisco CA']

In [ ]:
sf_test.plot()

In [ ]:
sf_test2 = final_df.loc[final_df.block.str.contains('^06075'), :]

In [ ]:
sf_test2.plot()

In [ ]:
df_joined['jobs_per_sq_meter'] = df_joined['total_jobs']/df_joined['land_area']

In [ ]:
df_joined.head()

In [ ]:
df_joined.land_area.mean()

In [ ]:
sanfran_chor = df_joined[df_joined['city']=='San Francisco CA']
sanfran_chor.head()

In [ ]:
sanfran_chor.explore(
    column="total_jobs",
    tooltip="total_jobs",
    tiles="CartoDB positron",
    style_kwds=dict(alpha=.5)
)

### Export as geodataframe

In [ ]:
df_joined.to_file("C:/Users/jpg23/data/downtownrecovery/lehd_new_downtowns/cities_lehd_jobs_block.geojson", driver="GeoJSON")  